In [9]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from unidecode import unidecode

import torch
import torch.nn as nn
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
import numpy as np

## ler arquivo json em pandas
df = pd.read_json(
    "/home/mfelipemota/projects/pantanaldev/data/label-studio/data/export/project-1-at-2023-04-30-05-08-956f4e3c.json"
)

df = df.drop(["id"], axis=1)
## expandir coluna annotations
df = pd.concat(
    [df.drop(["annotations"], axis=1), df["annotations"].apply(pd.Series)], axis=1
)
## expandir coluna 0 e renomear para annotations
df = pd.concat([df.drop([0], axis=1), df[0].apply(pd.Series)], axis=1)
## expandir coluna result e renomear para result
df = pd.concat([df.drop(["result"], axis=1), df["result"].apply(pd.Series)], axis=1)
df = df.drop(["id"], axis=1)

## expandir coluna 0 e renomear para result
df = pd.concat([df.drop([0], axis=1), df[0].apply(pd.Series)], axis=1)
## expandir coluna value e renomear para value
df = pd.concat([df.drop(["value"], axis=1), df["value"].apply(pd.Series)], axis=1)
## dropar choices nulos
df = df.dropna(subset=["choices"])
## obter choices
df["choices"] = df["choices"].apply(lambda x: x[0])

## expandir coluna data
df = pd.concat([df.drop(["data"], axis=1), df["data"].apply(pd.Series)], axis=1)

df_noticia_original = df.copy()


padrao_data_cepea = r"Cepea, \d{2}/\d{2}/\d{4} - "
df["noticia"] = df["noticia"].apply(lambda x: re.sub(padrao_data_cepea, "", x))

## remover a palavra 'cepea' das noticias
padrao_cepea = r"Cepea"
df["noticia"] = df["noticia"].apply(
    lambda x: re.sub(padrao_cepea, "", x, flags=re.IGNORECASE)
)

## remover numeros das noticias
padrao_numeros = r"[0-9]+"
df["noticia"] = df["noticia"].apply(lambda x: re.sub(padrao_numeros, "", x))

## noticia que contem a palavra 'soja'
df = df[df["titulo"].str.contains("soja", flags=re.IGNORECASE)]

## remover noticias com choice 'desclassificar'
df = df[df["choices"] != "Desclassificar"]
df.count()


file_upload                 236
drafts                      236
predictions                 236
meta                        236
created_at                  236
updated_at                  236
inner_id                    236
total_annotations           236
cancelled_annotations       236
total_predictions           236
comment_count               236
unresolved_comment_count    236
last_comment_updated_at       0
project                     236
updated_by                  236
comment_authors             236
completed_by                236
was_cancelled               236
ground_truth                236
created_at                  236
updated_at                  236
lead_time                   236
prediction                  236
result_count                236
unique_id                   236
last_action                   0
task                        236
project                     236
updated_by                  236
parent_prediction             0
parent_annotation             0
last_cre

In [10]:
# Selecionar apenas as colunas necessárias
columns_to_select = ['id', 'data', 'noticia', 'titulo', 'choices', 'unique_id']

df = df[columns_to_select]
df.dropna(subset=['noticia'])

# Pré-processamento dos dados
stop_words = set(stopwords.words('portuguese'))

def preprocess_text(text):
    # remover acentuação
    text = unidecode(text)
    # Remover pontuações
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenização
    words = word_tokenize(text.lower())
    # Remover stopwords
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

df['noticia'] = df['noticia'].apply(preprocess_text)

df_treino = df[:210]
df_validacao = df[210:]


In [11]:
from transformers import BertTokenizer, TFBertForSequenceClassification

# Carregar o modelo pré-treinado BERTimbau
model = TFBertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=3)
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)


ImportError: 
TFBertForSequenceClassification requires the TensorFlow library but it was not found in your environment.
However, we were able to find a PyTorch installation. PyTorch classes do not begin
with "TF", but are otherwise identically named to our TF classes.
If you want to use PyTorch, please use those classes instead!

If you really do want to use TensorFlow, please follow the instructions on the
installation page https://www.tensorflow.org/install that match your environment.


In [ ]:
model.summary()